[zillow-zestimate](https://www.zillow.com/zestimate/)

Notes:
`The Zestimate® home valuation model is Zillow’s estimate of a home's market value. The Zestimate incorporates public and user-submitted data, taking into account home facts, location and market conditions.`

`It is not an appraisal and it should be used as a starting point. We encourage buyers, sellers and homeowners to supplement the Zestimate with other research such as visiting the home, getting a professional appraisal of the home, or requesting a comparative market analysis (CMA) from a real estate agent.`

`The amount of data we have for your home and homes in your area directly affects the Zestimate's accuracy. If the data is incorrect or incomplete, update your home facts — this may affect your Zestimate.`    
    
`The Zestimate was created to give consumers more information about homes and the housing market. It is intended to provide user-friendly data to promote transparent real estate markets and allow people to make more informed decisions — it should not be used to drive up the price of a home. Zestimates are designed to track the market, not drive it.`
    
`The Zestimate is not an appraisal and can't be used in place of an appraisal. It is a computer-generated estimate of the value of a home today, given the available data.`

`Note: The Zestimate's accuracy is computed by comparing the final sale price to the Zestimate that was published on or just prior to the sale date.`

`Median Error:
    The nationwide median error rate for the Zestimate for on-market homes is 1.9%, while the Zestimate for off-market homes has a median error rate of 7.5%. This means that the Zestimates for half of all on-market homes are within 2% of the selling price, and half are not. For most major markets, the Zestimate for on-market homes is within 10% of the final sale price more than 95% of the time.`

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from prepare import prepare_zillow

In [2]:
df = prepare_zillow()

/usr/local/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (36,37,62,66,67) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df.shape

(52439, 24)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52439 entries, 0 to 52438
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   parcelid               52439 non-null  int64   
 1   num_of_bedrooms        52439 non-null  category
 2   num_of_restrooms       52439 non-null  category
 3   living_room_area_sqft  52439 non-null  float64 
 4   fips                   52439 non-null  int64   
 5   latitude               52439 non-null  float64 
 6   longitude              52439 non-null  float64 
 7   num_of_rooms           52439 non-null  category
 8   year_built             52439 non-null  int64   
 9   structure_tax          52439 non-null  float64 
 10  taxable_value          52439 non-null  float64 
 11  land_tax               52439 non-null  float64 
 12  property_tax           52439 non-null  float64 
 13  logerror               52439 non-null  float64 
 14  date_sold              52439 non-null 